In [1]:
#Import dependencies for Q learning

import numpy as np
import math
import random
import matplotlib.pyplot as plt #for plotting functions
#from DigiCommPy.passband_modulations import bpsk_mod, bpsk_demod
#from DigiCommPy.channels import awgn
from scipy.special import erfc
from numpy import sum,isrealobj,sqrt
from numpy.random import standard_normal

In [2]:
#Import dependencies for BackComm
#import numpy as np #for numerical computing
import matplotlib.pyplot as plt #for plotting functions
#from DigiCommPy.passband_modulations import bpsk_mod, bpsk_demod
#from DigiCommPy.channels import awgn
from scipy.special import erfc

### Number of steps/time slots ###

In [3]:
#Number of steps each episode will have. 1 step = 1 time slot
nSteps = 100 #number of time steps, in each time step, AP chooses a new channel

### Functions for BackComm System ###

In [4]:
#Let's define the functions to be used for BackComm
def bpsk_mod(mu,L):
    from scipy.signal import upfirdn
    s_bb = upfirdn(h=[1]*L, x=2*mu-1, up = L) # NRZ encoder; upfirdn: Upsample, FIR filter, and downsample.
    t=np.arange(start = 0,stop = len(mu)*L) #discrete time base
    return (s_bb,t)
        
def bpsk_demod(r_bb,L):
    x = np.real(r_bb) # I arm
    x = np.convolve(x,np.ones(L)) # integrate for Tb duration (L samples)
    x = x[L-1:-1:L] # I arm - sample at every L
    ak_hat = (x > 0).transpose() # threshold detector
    return ak_hat

#Noise (AP to BD)
def noise(s,SNRdB,L=1):
    gamma = 10**(SNRdB/10) #SNR to linear scale
    if s.ndim==1: # if s is single dimensional vector
        P=L*sum(abs(s)**2)/len(s) #Actual power in the vector
    else: # multi-dimensional signals like MFSK
        P=L*sum(sum(abs(s)**2))/len(s) # if s is a matrix [MxN]
    N0=P/gamma # Find the noise spectral density
    if isrealobj(s): # check if input is real/complex object type
        n = sqrt(N0/2)*standard_normal(s.shape) # computed noise
    else:
        n = sqrt(N0/2)*(standard_normal(s.shape)+1j*standard_normal(s.shape))
    #r = s + n # received signal
    return n

#Noise (BD to AP)
def awgn(s,SNRdB,L=1):
    gamma = 10**(SNRdB/10) #SNR to linear scale
    if s.ndim==1: # if s is single dimensional vector
        P=L*sum(abs(s)**2)/len(s) #Actual power in the vector
    else: # multi-dimensional signals like MFSK
        P=L*sum(sum(abs(s)**2))/len(s) # if s is a matrix [MxN]
    N0=P/gamma # Find the noise spectral density
    if isrealobj(s): # check if input is real/complex object type
        n = sqrt(N0/2)*standard_normal(s.shape) # computed noise
    else:
        n = sqrt(N0/2)*(standard_normal(s.shape)+1j*standard_normal(s.shape))
    r = s + n # received signal #AWGN channel
    return r
# In theoretical analyses and simulations, the channel gain ℎ is often considered as a complex Gaussian random variable 
# with unity magnitude (i.e.,|ℎ∣=1) and uniformly distributed phase between 0 and 2π. This assumption simplifies 
# the analysis and is a common model used in communication theory to characterize the effects of the channel.

### System Parameters ###

In [5]:
#System parameters
N=100 # Number of symbols to transmit
EbN0dB = np.arange(start=-4,stop = 11,step = (11+4)/nSteps) # Eb/N0 range in dB for simulation
L=16 # oversampling factor,L=Tb/Ts(Tb=bit period,Ts=sampling period)
deltaf = 10e6 #frequency step
Fc = 900e6 #900MHz #base carrier frequency
Fs=L*Fc # sampling frequency
BER = np.zeros(len(EbN0dB)) # for BER values for each Eb/N0
#Backscatter Reflection Coefficient - when the tag wants to transmits a 1, it reflects and absorbs when it wants to transmit a 0
mu = np.random.randint(2, size=N) # uniform random symbols from 0's and 1's
(s_bb,t)= bpsk_mod(mu,L) # BPSK modulation(waveform) - baseband


In [6]:
#Creating a jammer signal
def Jammer(f_k):
    K_J = 1 # 0 = no jammer present in current channel, 1 = jammer present in current channel
    h_J = 1.5e-5
    x_J = np.cos(2*np.pi*f_k*t/Fs)*h_J
    J = K_J * x_J #Jammer signal
    return J

In [7]:
#SIC
epsilon = 1e-10 #very small number
#g_hat: feedback channel coefficient
g_bar = 7e-5 #g_bar: feedback channel coefficient estimation
g_tilde = 6e-5#g_tilde: estimation error
g_hat = g_bar + (sqrt(epsilon))*g_tilde 

### State Space, Action Space ###

In [80]:
#Define State, Action

#Jammer
K_J = np.random.randint(low=0, high = 2, size=nSteps) #should be an array randomly changing between 0 and 1 of dimension = number of time steps
M = 5
P_J = K_J*(np.random.randint(low=1,high=M,size=nSteps)) #Absence of jammer in the channel indicates no jamming power; jamming power = 1-4 W
#Distribute the time slot in CS+EH+BC
r = 0.25 #duty cycle 25%
τ = 1 #duration of each time slot is 1s
τ_sense = 0.15
τ_switch = np.array([0.1 if val == 1 else 0 for val in K_J])
τ_EH = τ - (r*τ) - (τ_sense + τ_switch)
P_T = 1 #np.random.randint(low=0, high = M)
ξ = 0.8 #energy harvesting efficiency
U_E = ξ*(P_J + P_T)*τ_EH #Unit Energy #Signal energy doesn't depend upon frequency
E_BC = 0.75*U_E # Let 1 Unit energy used, fixed, by tag circuit during BC operation
E_EH = 0.25*U_E # Let 1 Unit energy used, fixed, by tag circuit during EH operation ### E_BC > E_EH
E_h = U_E #Amount of energy harvested #Energy harvested is function of time and power

    
#Reflection coeffcient
#s_bb = np.array([np.random.choice([1, -1]) for _ in range(nSteps)])
mu = np.array([1 if val == 1 else 0 for val in s_bb]) # uniform random symbols from 0's and 1's, when tag wants to transmit 0, mu = 0, when tag wants to transmit 1, mu = 1

E_B = np.zeros(nSteps)

for j in range(nSteps):
    E_B [j] = E_B [j] - mu[j]*E_BC[j] - (1 - mu[j])*E_EH[j] + (1 - mu[j])*E_h[j] + 1*U_E[j]

state_space = np.array([E_B,K_J]) #P_T doesn't contribute to channel selection decision #Channel/Frequency f_k has no effect on E_B or K_J
#define actions
#numeric action codes: 0 = channel 1, 1 = channel 2, 2 = channel 3, 3 = channel 4
action_space = np.array([0,1,2,3])

In [81]:
q_table = np.zeros((len(E_B), len(K_J), 4))
print(np.shape(q_table))
print(action_space)
print(q_table[0][0][:]) #print(test[0][0][:])

(100, 100, 4)
[0 1 2 3]
[0. 0. 0. 0.]


### Trainning Phase

In [82]:
#training parameters
num_episodes = 100 #total no. of episodes the agent will play during training
#In this case, number of steps per episode = number of time slots
#No terminating condition - each episode run up to "nSteps" - time slots
max_steps_per_episode = nSteps #one of the terminating condition, max no. steps in a single episode

learning_rate = 0.1 #high LR focus more on new, less on old; low LR learn nothing, use prior knowledge
discount_rate = 0.99 #high DR focus more on distant reward, low DR focus more on immediate reward

#epsilon-greedy
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.01

In [83]:
rewards_all_episodes = []
# Q-learning algorithm
for episode in range(num_episodes):
    current_state = (0,0)
    rewards_current_episode = 0 #no rewards at the beginning of each episode
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        p0 = random.uniform(0, 1)
        if p0 > exploration_rate:
            action = np.argmax(q_table[current_state][:]) 
            print('Exploited Action: ',action)
            #print(current_state)
            #raise SystemExit("Stop right there!")
        else:
            action = random.choice(action_space)
            print('Random Action: ',action)
        # Take new action - decide channel
        k = action #channel is decided based on e-greedy
        f_k = Fc + (k*deltaf)
        #print('AP uses channel '+str(k)+' having frequency '+str(f_k))
        carrier = np.cos(2*np.pi*f_k*t/Fs) #Signal send by AP
        y_BD = (carrier) + (Jammer(f_k)) + noise(carrier,EbN0dB[step],L) #J = K_J*x_J; received signal at BD
        s_BD = s_bb*y_BD # backscattered signal transmitted by tag; s_bb = 1 when mu =1, s_bb = -1 when mu =0
        # Compute received signal and add AWGN noise
        y_AP = awgn(s_BD,EbN0dB[step],L) # signal received at AP #h_k isn't separately considered as AWGN noise is considered
        y_AP_hat = y_AP + ((sqrt(epsilon))*g_tilde)*carrier  #signal received at AP after SIC #
        r_bb = y_AP_hat*np.cos(2*np.pi*f_k*t/Fs) # recovered baseband signal
        ak_hat = bpsk_demod(r_bb,L) # baseband correlation demodulator
        BER[step] = np.sum(mu!=ak_hat)/N # Bit Error Rate Computation 
        #rewards
        R_b = 10e3 #10kbps - tag transmits with a fixed data rate R_b
        τ = 1
        C = np.zeros(max_steps_per_episode)
        reward = np.zeros(max_steps_per_episode)
        C[step] = 1 + BER[step]*math.log(BER[step]) + (1 - BER[step])*math.log(1 - BER[step])
        reward[step] = R_b*C[step]*τ
        #next state
        next_row = np.random.randint(low=0, high = 2)
        next_column = np.random.randint(low=0, high = nSteps)
        new_state = (next_row,next_column)
        # Update Q-table for Q(s,a)
        q_table[current_state][action] = (1 - learning_rate) * q_table[current_state][action] + \
        learning_rate * (reward[step] + discount_rate * np.max(q_table[new_state][:]))
        #print(q_table)
        print('Run: ',step)
        # Transition to the next state
        current_state = new_state
        # Add new reward        
        rewards_current_episode += reward[step]
    # Exploration rate decay (at the end of one episode, we need to update the exploration rate)
    exploration_rate = min_exploration_rate + \
    (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)

# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/10)
#np.split: divides arrays into subarrays in N (where N = num_episodes/1000) equal arrays
count = 10
#print("********Average reward per thousand episodes********\n")
#for r in rewards_per_thousand_episodes:
#    print(count, ": ", str(sum(r/10)))
#    count += 10

Random Action:  3
Run:  0
Random Action:  2
Run:  1
Random Action:  2
Run:  2
Random Action:  3
Run:  3
Random Action:  3
Run:  4
Random Action:  0
Run:  5
Random Action:  2
Run:  6
Random Action:  3
Run:  7
Random Action:  1
Run:  8
Random Action:  3
Run:  9
Random Action:  1
Run:  10
Random Action:  3
Run:  11
Random Action:  2
Run:  12
Random Action:  0
Run:  13
Random Action:  3
Run:  14
Random Action:  1
Run:  15
Random Action:  3
Run:  16
Random Action:  0
Run:  17
Random Action:  3
Run:  18
Random Action:  2
Run:  19
Random Action:  1
Run:  20
Random Action:  3
Run:  21
Random Action:  2
Run:  22
Random Action:  2
Run:  23
Random Action:  1
Run:  24
Random Action:  2
Run:  25
Random Action:  0
Run:  26
Random Action:  0
Run:  27
Random Action:  0
Run:  28
Random Action:  2
Run:  29
Random Action:  3
Run:  30
Random Action:  0
Run:  31
Random Action:  2
Run:  32
Random Action:  0
Run:  33
Random Action:  3
Run:  34
Random Action:  2
Run:  35
Random Action:  0
Run:  36
Random Acti

C:\Users\Zunayeed\AppData\Local\Temp\ipykernel_18504\981917045.py:29: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  BER[step] = np.sum(mu!=ak_hat)/N # Bit Error Rate Computation


Run:  92
Random Action:  0
Run:  93
Random Action:  2
Run:  94
Random Action:  3
Run:  95
Random Action:  2
Run:  96
Random Action:  1
Run:  97
Random Action:  2
Run:  98
Exploited Action:  3
Run:  99
Random Action:  3
Run:  0
Random Action:  2
Run:  1
Random Action:  2
Run:  2
Random Action:  3
Run:  3
Random Action:  2
Run:  4
Random Action:  2
Run:  5
Random Action:  0
Run:  6
Random Action:  2
Run:  7
Random Action:  3
Run:  8
Random Action:  1
Run:  9
Random Action:  0
Run:  10
Random Action:  3
Run:  11
Random Action:  1
Run:  12
Random Action:  3
Run:  13
Random Action:  3
Run:  14
Random Action:  1
Run:  15
Random Action:  0
Run:  16
Random Action:  1
Run:  17
Random Action:  3
Run:  18
Random Action:  0
Run:  19
Random Action:  1
Run:  20
Random Action:  2
Run:  21
Random Action:  0
Run:  22
Random Action:  3
Run:  23
Random Action:  3
Run:  24
Random Action:  3
Run:  25
Random Action:  3
Run:  26
Random Action:  2
Run:  27
Random Action:  1
Run:  28
Random Action:  1
Run:  29

Run:  36
Random Action:  2
Run:  37
Random Action:  2
Run:  38
Random Action:  2
Run:  39
Random Action:  3
Run:  40
Random Action:  0
Run:  41
Random Action:  2
Run:  42
Random Action:  3
Run:  43
Random Action:  2
Run:  44
Random Action:  2
Run:  45
Random Action:  1
Run:  46
Random Action:  0
Run:  47
Random Action:  3
Run:  48
Random Action:  2
Run:  49
Random Action:  0
Run:  50
Random Action:  3
Run:  51
Random Action:  1
Run:  52
Random Action:  1
Run:  53
Random Action:  0
Run:  54
Random Action:  3
Run:  55
Random Action:  0
Run:  56
Random Action:  3
Run:  57
Random Action:  1
Run:  58
Random Action:  0
Run:  59
Random Action:  2
Run:  60
Random Action:  0
Run:  61
Random Action:  1
Run:  62
Random Action:  3
Run:  63
Random Action:  0
Run:  64
Random Action:  0
Run:  65
Random Action:  2
Run:  66
Random Action:  2
Run:  67
Random Action:  0
Run:  68
Random Action:  0
Run:  69
Random Action:  0
Run:  70
Random Action:  2
Run:  71
Random Action:  1
Run:  72
Random Action:  2
R

Run:  65
Random Action:  2
Run:  66
Random Action:  0
Run:  67
Random Action:  0
Run:  68
Random Action:  3
Run:  69
Random Action:  3
Run:  70
Random Action:  2
Run:  71
Random Action:  0
Run:  72
Random Action:  2
Run:  73
Random Action:  0
Run:  74
Random Action:  1
Run:  75
Random Action:  2
Run:  76
Random Action:  1
Run:  77
Random Action:  0
Run:  78
Exploited Action:  3
Run:  79
Random Action:  2
Run:  80
Random Action:  0
Run:  81
Random Action:  1
Run:  82
Random Action:  1
Run:  83
Random Action:  0
Run:  84
Exploited Action:  1
Run:  85
Random Action:  1
Run:  86
Random Action:  2
Run:  87
Random Action:  2
Run:  88
Random Action:  1
Run:  89
Random Action:  2
Run:  90
Random Action:  2
Run:  91
Random Action:  3
Run:  92
Random Action:  0
Run:  93
Random Action:  2
Run:  94
Random Action:  2
Run:  95
Random Action:  3
Run:  96
Random Action:  2
Run:  97
Random Action:  2
Run:  98
Random Action:  0
Run:  99
Random Action:  3
Run:  0
Random Action:  2
Run:  1
Random Action: 

...

Run:  53
Exploited Action:  0
Run:  54
Exploited Action:  3
Run:  55
Exploited Action:  3
Run:  56
Random Action:  2
Run:  57
Random Action:  1
Run:  58
Random Action:  2
Run:  59
Random Action:  2
Run:  60
Exploited Action:  3
Run:  61
Random Action:  3
Run:  62
Exploited Action:  1
Run:  63
Random Action:  1
Run:  64
Exploited Action:  3
Run:  65
Exploited Action:  3
Run:  66
Exploited Action:  0
Run:  67
Random Action:  1
Run:  68
Random Action:  1
Run:  69
Exploited Action:  2
Run:  70
Random Action:  1
Run:  71
Exploited Action:  3
Run:  72
Exploited Action:  3
Run:  73
Exploited Action:  2
Run:  74
Random Action:  3
Run:  75
Random Action:  2
Run:  76
Exploited Action:  3
Run:  77
Exploited Action:  2
Run:  78
Exploited Action:  3
Run:  79
Exploited Action:  0
Run:  80
Exploited Action:  0
Run:  81
Random Action:  3
Run:  82
Exploited Action:  1
Run:  83
Exploited Action:  1
Run:  84
Random Action:  2
Run:  85
Exploited Action:  1
Run:  86
Exploited Action:  1
Run:  87
Random Act

Run:  41
Exploited Action:  1
Run:  42
Exploited Action:  2
Run:  43
Exploited Action:  2
Run:  44
Random Action:  1
Run:  45
Random Action:  1
Run:  46
Random Action:  2
Run:  47
Random Action:  2
Run:  48
Exploited Action:  3
Run:  49
Random Action:  2
Run:  50
Exploited Action:  1
Run:  51
Exploited Action:  0
Run:  52
Random Action:  3
Run:  53
Exploited Action:  2
Run:  54
Exploited Action:  0
Run:  55
Exploited Action:  3
Run:  56
Exploited Action:  0
Run:  57
Random Action:  3
Run:  58
Exploited Action:  2
Run:  59
Exploited Action:  1
Run:  60
Exploited Action:  2
Run:  61
Random Action:  3
Run:  62
Exploited Action:  0
Run:  63
Exploited Action:  0
Run:  64
Random Action:  0
Run:  65
Random Action:  0
Run:  66
Random Action:  0
Run:  67
Exploited Action:  2
Run:  68
Exploited Action:  2
Run:  69
Exploited Action:  3
Run:  70
Exploited Action:  3
Run:  71
Exploited Action:  3
Run:  72
Random Action:  3
Run:  73
Random Action:  1
Run:  74
Random Action:  2
Run:  75
Exploited Act

Run:  54
Random Action:  3
Run:  55
Random Action:  3
Run:  56
Exploited Action:  0
Run:  57
Random Action:  3
Run:  58
Random Action:  3
Run:  59
Random Action:  1
Run:  60
Random Action:  0
Run:  61
Random Action:  1
Run:  62
Random Action:  0
Run:  63
Exploited Action:  3
Run:  64
Exploited Action:  3
Run:  65
Exploited Action:  2
Run:  66
Exploited Action:  3
Run:  67
Random Action:  2
Run:  68
Exploited Action:  3
Run:  69
Exploited Action:  3
Run:  70
Random Action:  0
Run:  71
Random Action:  1
Run:  72
Exploited Action:  1
Run:  73
Random Action:  0
Run:  74
Exploited Action:  3
Run:  75
Random Action:  0
Run:  76
Random Action:  3
Run:  77
Random Action:  3
Run:  78
Random Action:  1
Run:  79
Exploited Action:  3
Run:  80
Exploited Action:  0
Run:  81
Random Action:  3
Run:  82
Random Action:  2
Run:  83
Random Action:  1
Run:  84
Random Action:  1
Run:  85
Exploited Action:  0
Run:  86
Random Action:  2
Run:  87
Exploited Action:  0
Run:  88
Exploited Action:  1
Run:  89
Rand

Run:  76
Exploited Action:  0
Run:  77
Exploited Action:  3
Run:  78
Random Action:  2
Run:  79
Exploited Action:  3
Run:  80
Exploited Action:  2
Run:  81
Exploited Action:  1
Run:  82
Exploited Action:  3
Run:  83
Random Action:  0
Run:  84
Random Action:  3
Run:  85
Random Action:  0
Run:  86
Exploited Action:  2
Run:  87
Exploited Action:  2
Run:  88
Exploited Action:  3
Run:  89
Exploited Action:  0
Run:  90
Exploited Action:  0
Run:  91
Exploited Action:  2
Run:  92
Exploited Action:  1
Run:  93
Exploited Action:  1
Run:  94
Exploited Action:  0
Run:  95
Random Action:  0
Run:  96
Random Action:  2
Run:  97
Exploited Action:  3
Run:  98
Random Action:  1
Run:  99
Random Action:  2
Run:  0
Exploited Action:  3
Run:  1
Exploited Action:  0
Run:  2
Random Action:  2
Run:  3
Exploited Action:  0
Run:  4
Random Action:  2
Run:  5
Random Action:  2
Run:  6
Exploited Action:  0
Run:  7
Exploited Action:  3
Run:  8
Random Action:  3
Run:  9
Random Action:  3
Run:  10
Exploited Action:  0

Exploited Action:  1
Run:  73
Random Action:  3
Run:  74
Exploited Action:  2
Run:  75
Exploited Action:  1
Run:  76
Exploited Action:  1
Run:  77
Exploited Action:  3
Run:  78
Random Action:  0
Run:  79
Exploited Action:  2
Run:  80
Exploited Action:  3
Run:  81
Exploited Action:  3
Run:  82
Random Action:  0
Run:  83
Exploited Action:  2
Run:  84
Exploited Action:  2
Run:  85
Random Action:  1
Run:  86
Exploited Action:  0
Run:  87
Exploited Action:  0
Run:  88
Random Action:  2
Run:  89
Exploited Action:  1
Run:  90
Exploited Action:  0
Run:  91
Exploited Action:  3
Run:  92
Random Action:  2
Run:  93
Random Action:  2
Run:  94
Random Action:  1
Run:  95
Exploited Action:  3
Run:  96
Exploited Action:  1
Run:  97
Exploited Action:  1
Run:  98
Exploited Action:  3
Run:  99


In [78]:
action = np.argmax(q_table[current_state,:])
print(current_state, '\n', action)
print(q_table[1][56][:])
#print(q_table[current_state][:])
#print(action_2)

(1, 56) 
 0
[0. 0. 0. 0.]


In [ ]:
import numpy as np    
A = np.matrix([[1,2,3,33],[4,5,6,66],[7,8,9,99]])

np.argmax(A)  # 11, which is the position of 99

all_r_c = np.argmax(A[:,:])  # 11, which is the position of 99
print('all_r_c: ',all_r_c)

second_row = np.argmax(A[1,:])  # 3, which is the position of 33
print('second_row: ',second_row)

third_column = np.argmax(A[:,2])  # 2, which is the position of 9
print('third_column: ',third_column )


print(np.shape(A),'\n',A)

In [ ]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)

In [ ]:
BER[step] = np.sum(mu!=ak_hat)/N
print(mu)
print(ak_hat)
print(BER)
print(np.shape(mu),np.shape(ak_hat),np.shape(BER))

## Plots ##

In [ ]:
#plotting the baseband signal
plt.figure(figsize=(10, 4))
plt.plot(t,s_bb) # baseband wfm zoomed to first 10 bits
plt.xlabel('time (in s)')
plt.xlim(0,10*L);
plt.ylabel(r'$s_{bb}(t)$-baseband')
plt.grid(True)
plt.show()

#signal constellation at transmitter
plt.plot(np.real(s_bb),np.imag(s_bb),'o')
plt.xlim(-1.5,1.5) 
plt.ylim(-1.5,1.5)
plt.title('Constellation diagram showing 2 symbols')
plt.grid(True)
plt.show()

#plotting the jammer signal
plt.plot(t,Jammer(f_k)) # transmitted wfm zoomed to first 10 bits
plt.xlabel('time (in s)') 
plt.xlim(0,10*L)
plt.ylabel('Jammer signal')
plt.grid(True)
plt.show()

#plotting the bandpass signal
plt.plot(t,s_BD) # transmitted wfm zoomed to first 10 bits
plt.xlabel('time (in s)') 
plt.xlim(0,10*L)
plt.ylabel('Backscattered signal')
plt.grid(True)
plt.show()

# Received signal waveform zoomed to first 10 bits
plt.plot(t,y_AP) # received signal (with noise)
plt.xlabel('time (in s)')
plt.ylabel('Received signal (t)')
plt.xlim(0,10*L)
plt.grid(True)
plt.show()

#------Theoretical Bit/Symbol Error Rates-------------
theoreticalBER = 0.5*erfc(np.sqrt(10**(EbN0dB/10))) # BER = (1/2)*erfc(sqrt(Eb/N0)) = Q(sqrt(2Eb/N0))
#-------------Plots---------------------------
fig1, ax1 = plt.subplots(nrows=1,ncols = 1)
ax1.semilogy(EbN0dB,BER,'k*',label='Simulated') # simulated BER
ax1.semilogy(EbN0dB,theoreticalBER,'r*',label='Theoretical')
ax1.set_xlabel(r'$E_b/N_0$ (dB)')
ax1.set_ylabel(r'Probability of Bit Error - $P_b$')
ax1.set_title('Probability of Bit Error for BPSK modulation')
plt.grid(True)